# RAG のための前処理

指定ディレクトリ内のすべての .txt ファイルを読み込み、NLTK を使ってテキストを文ごとに分割し、指定した文数ごとにチャンク化した上で、各チャンクの埋め込みを生成して出力します。

split_text_into_chunks 関数
NLTK の sent_tokenize を使用してテキストを文単位に分割し、max_sentences 毎に結合してチャンク化しています。

get_embedding 関数
最新の OpenAI API では、input パラメータにテキストのリストを渡す必要があるため、input=[text] としています。
生成された応答から、最初のチャンクの埋め込みを取り出して返します。

main 関数
指定ディレクトリ内の全 .txt ファイルを読み込み、各ファイルのテキストをチャンクに分割し、各チャンクに対して埋め込みを生成、最初の5次元だけを出力しています。

このコードを実行すれば、まずは小さなサンプルテキストを使った前処理と埋め込み生成の流れを体験できます。
その後、生成した埋め込みを Pinecone や Weaviate などのベクトルデータベースに保存するなど、RAG システムの次のステップに進むことができます。

### テキストの分割・前処理
- 長い文章は、意味のまとまりごとに分割する必要があります。

- たとえば、「文章を文単位に分割し、さらにいくつかの文ごとにまとめてチャンクを作る」方法が考えられます。

# 🔧 これまでのステップ：RAG構築の準備プロセス
### 1. Pineconeのセットアップ
- PineconeのWebコンソールにログインし、新しいプロジェクト・インデックス（hamster-embeddings）を作成。

- 設定内容：
    - Dimension: 1536（OpenAIのtext-embedding-ada-002に対応）
    - Metric: cosine
    - Type: Dense
    - Capacity Mode: Serverless
    - Cloud: AWS、Region: us-east-1

### 2. Docker環境の構築
- Jupyter環境でNotebookを使ってRAG処理を行うため、DockerでPython+Jupyterの環境を構築。

- requirements.txtに以下の主要ライブラリを指定：
    - openai>=1.0.0
    - pinecone-client==2.2.4
    - httpx
    - nltk
    - jupyter

- Dockerfileでは追加で build-essential, ca-certificates, curl などもインストールし、通信や証明書の問題を回避。

### 3. テキストデータの分割とベクトル化
- senarios/ ディレクトリ内の .txt ファイルを読み込み、NLTKの sent_tokenize を使って文単位でチャンクに分割。

- OpenAIの text-embedding-ada-002 モデルを使って、各チャンクの文章を1536次元の埋め込みベクトルに変換。

### 4. Pineconeへのアップロード（Upsert）
- pinecone.Index.upsert() により、ベクトルとメタデータ（元のテキスト）をPineconeに保存。

- 実行成功後、Pineconeのダッシュボードで Record Count: 11 を確認（→成功！🎉）

In [2]:
import socket

try:
    print("🔍 解決結果:", socket.gethostbyname("hamster-embeddings-4ak3maz.svc.us-east-1-aws.pinecone.io"))
except Exception as e:
    print("❌ 名前解決できませんでした:", e)


🔍 解決結果: 34.230.203.60


In [3]:
print('hello, http://127.0.0.1:8888/tree')

hello, http://127.0.0.1:8888/tree


In [4]:
from openai import OpenAI
from pinecone import Pinecone
from nltk.tokenize import sent_tokenize
import nltk
import os
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [5]:
# OpenAI クライアントの初期化
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# Pinecone クライアントの初期化
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))

# インデックスに接続
index = pc.Index("hamster-embeddings")

# コンテナのディレクトリ
directory = "./senarios"

In [6]:
# --- ユーティリティ関数 ---
def split_text_into_chunks(text, max_sentences=2):
    sentences = sent_tokenize(text)
    return [" ".join(sentences[i:i+max_sentences]) for i in range(0, len(sentences), max_sentences)]

def get_embedding(text):
    res = client.embeddings.create(input=[text], model="text-embedding-ada-002")
    return res.data[0].embedding

In [7]:
# チャンクと埋め込みの生成（これで chunks と embeddings が作られます）
chunks = []

for filename in os.listdir(directory):
    if filename.endswith(".txt"):
        with open(os.path.join(directory, filename), 'r', encoding='utf-8') as f:
            text = f.read()
        chunks.extend(split_text_into_chunks(text))

# 埋め込みの生成
embeddings = [get_embedding(chunk) for chunk in chunks]


In [9]:
# Pinecone へアップロード（今のエラーが出たセル）

vectors = []
for i, embedding in enumerate(embeddings):
    vector_id = f"chunk{i+1}"
    metadata = {"text": chunks[i]}
    vectors.append((vector_id, embedding, metadata))

# ベクトルを Pinecone にアップロード
index.upsert(vectors=vectors)

print("✅ Embeddings uploaded successfully to Pinecone!")


✅ Embeddings uploaded successfully to Pinecone!


# 🧠 次に目指すステップ
以下のような処理を実装していくことで、RAGの完全動作へと進化できます。

## ✅ 今後のステップ案
### 1.検索（ベクトル検索）機能の実装

- クエリテキストを入力し、その埋め込みベクトルを生成。
- Pineconeで類似ベクトルを検索し、最も近いテキストを取得。

### 2.OpenAIへのプロンプト生成

- 検索結果から得たチャンクを組み合わせて「コンテキスト付きプロンプト」を構築。
- ChatGPTにそのプロンプトを送信し、回答を得る。

### 3.ユーザーインターフェース（例：Streamlit or Flask）
- ユーザーがクエリを入力し、検索〜応答生成を体験できるUIを構築。

### 4.チャットの履歴保存や拡張（任意）
- 会話履歴を保存、マルチターンの対話なども視野に。